# Gerando Visualizações com Plotly e Integrando Cloud Object Storage e Apache Spark com ibmos2spark

### 1 - Setup Inicial e Configuração de Credenciais

#### 1.1 - Importação de dependências e inicialização do Spark

In [51]:
# Importação de algumas dependências

from IPython.display import Image, HTML, display
import plotly as py
import plotly.graph_objs as go
import pandas as pd
import requests
requests.packages.urllib3.disable_warnings()

In [52]:
# Inicialização do Spark Context

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#### 1.2 - Configuração de credenciais do pacote Plotly

In [53]:
# Escreva suas credenciais Plotly na função abaixo

py.tools.set_credentials_file(username='<INSIRA SEU USERNAME AQUI>', api_key='<INSIRA SUA API-KEY AQUI>')

#### 1.3 - Inserção dos dados do arquivo .csv no Cloud Object Storage em um DataFrame Spark

In [ ]:
# Clique nesta célula;
# Clique em "Find and add Data" (canto superior direito);
# Escolha o arquivo .csv (enem2016data.csv1.csv) e selecione "Insert as SparkSession Dataframe".

'<INSIRA O CÓDIGO GERADO AQUI>'

### 2 - Preparação dos Dados

In [17]:
# Seleção das colunas desejadas

df = df_data_2.select('TP_SEXO', 'Q001', 'Q002', 'TP_ESCOLA', 'Q006', 'NU_NOTA_REDACAO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT')
df.show(5)

+-------+----+----+---------+----+---------------+----------+----------+----------+----------+
|TP_SEXO|Q001|Q002|TP_ESCOLA|Q006|NU_NOTA_REDACAO|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|
+-------+----+----+---------+----+---------------+----------+----------+----------+----------+
|      M|   E|   E|        1|   B|            580|       550|     629.4|       574|     462.9|
|      M|   H|   H|        1|   D|            560|     576.6|     669.8|     610.1|     423.6|
|      M|   B|   E|        1|   D|            480|     485.8|     576.9|     637.4|     414.6|
|      M|   E|   E|        1|   B|            620|     571.4|     628.6|     646.2|     709.4|
|      F|   A|   A|        1|   A|           null|      null|      null|      null|      null|
+-------+----+----+---------+----+---------------+----------+----------+----------+----------+
only showing top 5 rows



In [18]:
# Remoção de linhas com dados faltantes

print("Rows: ", df.count())
df = df.dropna()
print("Rows: ", df.count())

Rows:  9999
Rows:  7201


In [20]:
# Conversão de Spark DataFrames para Pandas DataFrames (para utilizar o pacote Plotly)

sqlCtx.registerDataFrameAsTable(df, "DFasTABLE")
df_rd = sqlCtx.sql("SELECT NU_NOTA_REDACAO as rd from DFasTABLE")
df_cn = sqlCtx.sql("SELECT NU_NOTA_CN as cn from DFasTABLE")
df_mt = sqlCtx.sql("SELECT NU_NOTA_MT as mt from DFasTABLE")
df_ch = sqlCtx.sql("SELECT NU_NOTA_CH as ch from DFasTABLE")
df_lc = sqlCtx.sql("SELECT NU_NOTA_LC as lc from DFasTABLE")
data_rd = [go.Histogram(x = df_rd.toPandas()['rd'])]
data_cn = [go.Histogram(x = df_cn.toPandas()['cn'])]
data_mt = [go.Histogram(x = df_mt.toPandas()['mt'])]
data_ch = [go.Histogram(x = df_ch.toPandas()['ch'])]
data_lc = [go.Histogram(x = df_lc.toPandas()['lc'])]

### 3 - Geração de Visualizações: Histogramas e Pie Charts

In [23]:

layout = go.Layout(
    title = "Histograma: notas da prova de redação ENEM 2016",
    xaxis = dict(
        range = [0, 1000]
    )
)

fig = go.Figure(data = data_rd, layout = layout)
py.plotly.iplot(fig, filename="enem2016_redacao")

In [24]:
layout = go.Layout(
    title = "Histograma: notas da prova de ciências da natureza ENEM 2016",
    xaxis = dict(
        range = [0, 1000]
    )
)

fig = go.Figure(data = data_cn, layout = layout)
py.plotly.iplot(fig, filename="enem2016_ciencias_da_natureza")

In [25]:
layout = go.Layout(
    title = "Histograma: notas da prova de matemática ENEM 2016",
    xaxis = dict(
        range = [0, 1000]
    )
)

fig = go.Figure(data = data_mt, layout = layout)
py.plotly.iplot(fig, filename="enem2016_matematica")

In [26]:
layout = go.Layout(
    title = "Histograma: notas da prova de ciências humanas ENEM 2016",
    xaxis = dict(
        range = [0, 1000]
    )
)

fig = go.Figure(data = data_ch, layout = layout)
py.plotly.iplot(fig, filename="enem2016_ciencias_humanas")

In [27]:
layout = go.Layout(
    title = "Histograma: notas da prova de linguagem e comunicação ENEM 2016",
    xaxis = dict(
        range = [0, 1000]
    )
)

fig = go.Figure(data = data_lc, layout = layout)
py.plotly.iplot(fig, filename="enem2016_linguagem_e_comunicacao")

In [28]:
df.cube("Q006").count().show()

+----+-----+
|Q006|count|
+----+-----+
|   K|   82|
|   I|  250|
|   G|  536|
|   P|   60|
|   B| 1264|
|null| 7201|
|   D| 1021|
|   O|   82|
|   L|   45|
|   A|  138|
|   H|  394|
|   E|  740|
|   J|  129|
|   F|  484|
|   Q|   68|
|   C| 1759|
|   N|   95|
|   M|   54|
+----+-----+



In [29]:
labels = ['1 SM','2 SM','3 SM','4 SM','5 SM','6 SM','7 SM','8 SM','9 SM','10 SM','11 SM','12 SM','13 SM','14 SM','15 SM','16 SM','17 SM']
values = [138,1264,1759,1021,740,484,536,394,250,129,82,45,54,95,82,60,68]

trace = go.Pie(labels = labels, values = values)

py.plotly.iplot([trace], filename='basic_pie_chart')

In [31]:
df_q1 = sqlCtx.sql("SELECT Q001 as q1 from DFasTABLE")
df_q2 = sqlCtx.sql("SELECT Q002 as q2 from DFasTABLE")

trace1 = go.Histogram(
    x=df_q1.toPandas()['q1'],
    opacity=0.3,
    name="Escolaridade do Pai"
)
trace2 = go.Histogram(
    x=df_q2.toPandas()['q2'],
    opacity=0.3,
    name="Escolaridade da Mãe"
)

data = [trace1, trace2]
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=data, layout=layout)

py.plotly.iplot(fig, filename='overlaid histogram')

In [33]:
labels = ['Homens', 'Mulheres']
values = [2906, 4295]

trace = go.Pie(labels = labels, values = values)

py.plotly.iplot([trace], filename='basic_pie_chart2')